In [ ]:
import pystac_client
from odc.geo import BoundingBox

In [ ]:
client = pystac_client.Client.open("http://127.0.0.1:9588")
client

In [ ]:
import folium
from toolz.functoolz import curry

In [ ]:
categories = {
    "bo": "#008888",
    "db": "#880000",
    "ml": "#880088",
    "mo": "#888800",
    "pf": "#000088",
    "rf": "#008800",
    "tg": "#ff88ff",
    "ts": "#81d4fa",
}


def style_function(_, category):
    fill_color = categories.get(category, "#000000")
    style = {
        # "fillColor": fill_color,
        "color": fill_color,
    }

    return {k: v for k, v in style.items() if v is not None}

In [ ]:
def extract_category(id):
    return id.split("-")[-1].lower()

In [ ]:
def visualize_item(item):
    category = extract_category(item.collection_id)

    marker = folium.CircleMarker()
    return folium.GeoJson(
        item.geometry,
        style_function=curry(style_function, category=category),
        marker=marker,
    )

In [ ]:
def visualize_search(extent, items):
    m = folium.Map(width="80%", height="80%", tiles="cartodbpositron")

    folium.GeoJson(extent.boundary().geojson()).add_to(m)
    for item in items:
        visualize_item(item).add_to(m)

    return m

In [ ]:
extent = BoundingBox(-15, 43, 5, 50)
items = client.search(
    datetime="2022-06-01/2022-06-30", bbox=extent.bbox
).item_collection()
items

In [ ]:
import xarray as xr
import fsspec

In [ ]:
asset = items[0].assets["https"]
fs = fsspec.filesystem("http")
xr.open_dataset(fs.open(asset.href), **asset.extra_fields["xarray:open_kwargs"])